In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import time
import glob
import os
from datetime import datetime


In [2]:
df = pd.read_csv(r"E:/all - fivedataall.csv") 
del df['Unnamed: 0']

In [3]:
df.head()

,time,exstock,fivemiu,fivestd,countobs,ym,ymd,miumiu,stkcd
0,2020-11-25 09:34:58,SH000906,-0.000355,0.001070,48,202011,20201125,-0.017030,906
1,2020-11-26 09:34:57,SH000906,0.000027,0.001157,48,202011,20201126,0.001273,906
2,2020-11-27 09:35:00,SH000906,0.000166,0.000975,48,202011,20201127,0.007961,906
3,2020-11-30 09:34:56,SH000906,-0.000098,0.001418,48,202011,20201130,-0.004694,906
4,2020-12-01 09:34:58,SH000906,0.000412,0.000979,48,202012,20201201,0.019772,906


In [4]:
rangemax=0.08   #自己设置   对个股设置0.08，对指数设置0.06
w=0.002

In [5]:
#如果数据量太大可能跑不动，可以筛选一下数据，分几次跑

dataraw1=df
index = dataraw1.loc[(dataraw1['fivestd']==0)].index
dataraw=dataraw1.drop(index)
dataraw = dataraw[(dataraw['countobs']>14)]
dataraw.dropna(axis=0, how="any",inplace=True)
print(len(dataraw1)-len(dataraw))

data =dataraw
print(data.head(2))
miu=np.array(data.fivemiu)*np.array(data.countobs)
sigma=np.sqrt(np.array(data.countobs))*np.array(data.fivestd)
n=miu.size
w=0.002
grid=np.arange(-rangemax-w,rangemax+2*w,w)
grid[0]=float('-inf')
grid[-1]=float('inf')
for j in range (grid.size-1):
        data.loc[:,"prob"+str(j)] = norm.cdf(grid[j+1],miu,sigma)-norm.cdf(grid[j],miu,sigma)
data["cate"] = data["exstock"].map(str) + data["ym"].map(str)
vv=data.drop(['time','ymd','exstock','fivemiu','fivestd','countobs','ym'],axis = 1,inplace = False)
print(vv.head(2))
grouped=vv.groupby('cate')
AA=grouped.mean()
BB=grouped.var()
CC=grouped.count()
print(AA.head(2))
print(BB.head(2))
a=np.matrix(AA)
b=np.matrix(BB)
coef=1/(w*(1-w))
amb=coef*np.sum(np.multiply(a,b),axis=1)
amb2=pd.DataFrame(data=amb,columns=['ambiguity'])
timearr=np.array(AA.index)
time2=pd.DataFrame(data=timearr,columns=['time'])
count=np.array(CC.prob0)
count2=pd.DataFrame(data=count,columns=['countobs'])
monthlyamb=pd.concat([time2,amb2,count2],axis=1)
print(monthlyamb.size/3)
#monthlyamb=monthlyamb[(monthlyamb['countobs']>14)]

ab=monthlyamb
def month1(hang):
    return hang['time'][2:8]
ab['stock']=ab.apply(month1,axis=1)
def month2(hang):
    return hang['time'][0:8]
ab['exstock']=ab.apply(month2,axis=1)
def month3(hang):
    return (int(hang['time'][8:12])-2007)*12+int(hang['time'][12:14])-1
ab['date1']=ab.apply(month3,axis=1)
def month4(hang):
    return int(hang['time'][8:12])
ab['year']=ab.apply(month4,axis=1)
def month5(hang):
    return int(hang['time'][12:14])
ab['month']=ab.apply(month5,axis=1)
ab.rename(columns={'date1': 'monthnumber'}, inplace=True)

0
                  time   exstock   fivemiu   fivestd  countobs      ym  \
0  2020-11-25 09:34:58  SH000906 -0.000355  0.001070        48  202011   
1  2020-11-26 09:34:57  SH000906  0.000027  0.001157        48  202011   

        ymd    miumiu  stkcd  
0  20201125 -0.017030    906  
1  20201126  0.001273    906  
     miumiu  stkcd         prob0         prob1         prob2         prob3  \
0 -0.017030    906  9.880502e-18  8.741206e-17  7.943558e-16  6.714209e-15   
1  0.001273    906  1.872487e-24  2.145985e-23  2.500232e-22  2.737849e-21   

          prob4         prob5         prob6         prob7  ...        prob73  \
0  5.278529e-14  3.859856e-13  2.625257e-12  1.660798e-11  ...  0.000000e+00   
1  2.817837e-20  2.725843e-19  2.478366e-18  2.117929e-17  ...  2.220446e-15   

         prob74  prob75  prob76  prob77  prob78  prob79  prob80  prob81  \
0  0.000000e+00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1  3.330669e-16     0.0     0.0     0.0     0.0     0.0 

In [6]:
strrange='%d'%(rangemax*100)

ab.to_csv(r'E:/monthlyamb '+strrange+'pct.csv',index=None)